In [1]:
from pathlib import Path
from pprint import pprint

import pandas as pd
import numpy as np

from bayes_opt import BayesianOptimization

from isic_helper import get_folds
from isic_helper import compute_pauc, compute_auc

In [2]:
id_column = "isic_id"
target_column = "target"
ensemble_column = "oof_preds_ensemble"

# model_names = ["cb", "lgb", "resnet18", "efficientnet_b0"]
# versions = ["v1", "v3", "v2",  "v1"]
model_names = ["cb", "lgb", "resnet18"]
versions = ["v1", "v3", "v5"]
paths = [f"/kaggle/input/isic-scd-{model_name.replace('_', '-')}-{version}-train" for model_name, version in zip(model_names, versions)]

In [3]:
val_auc_scores = {}
val_pauc_scores = {}
for idx, path in enumerate(paths):
    model_name = model_names[idx]
    version = versions[idx]
    oof_preds_model_df = pd.read_csv(f"{path}/oof_preds_{model_name}_{version}.csv")
    if idx == 0:
        oof_preds_df = oof_preds_model_df.copy()
    else:
        oof_preds_df = oof_preds_df.merge(oof_preds_model_df[[id_column, f"oof_{model_name}_{version}"]],
                                          on="isic_id", how="inner")
        assert oof_preds_df.shape[0] == oof_preds_model_df.shape[0]
    val_auc_scores[f"{model_name}_{version}"] = {}
    val_pauc_scores[f"{model_name}_{version}"] = {}
val_auc_scores["ensemble"] = {}
val_pauc_scores["ensemble"] = {}

In [4]:
oof_columns = [col for col in oof_preds_df.columns if col.startswith("oof")]
all_folds = np.unique(oof_preds_df["fold"])

In [5]:
def blend_optimizer(oof_preds_df, oof_columns, init_points = 20, n_iter = 100):
    pbounds = {f"w{i}": (0.001, 10.0) for i in range(len(oof_columns))}

    def dim_opt(oof_preds_df, *args):
        weights = args
        score = 0
        for fold in all_folds:
            fold_ensemble_preds = 0
            for weight, oof_column in zip(weights, oof_columns):
                fold_ensemble_preds += weight * oof_preds_df.loc[oof_preds_df["fold"] == fold, oof_column].rank(pct=True).values
            score += compute_pauc(np.array(oof_preds_df.loc[oof_preds_df["fold"] == fold, "target"]), fold_ensemble_preds)
        return score / len(all_folds)
    
#     def dim_opt(oof_preds_df, *args):
#         weights = args
#         score = 0
#         for fold in all_folds:
#             oof_fold_preds_df = oof_preds_df[oof_preds_df["fold"] == fold]
#             fold_ensemble_preds = np.zeros((oof_fold_preds_df.shape[0], len(weights)))
#             for idx, (weight, oof_column) in enumerate(zip(weights, oof_columns)):
#                 fold_ensemble_preds[:, idx] = weight * np.log1p(oof_fold_preds_df[oof_column].rank(pct=True).values)
#             fold_ensemble_preds = np.exp(fold_ensemble_preds.sum(axis=1) / sum(weights)) - 1
#             score += compute_pauc(oof_fold_preds_df["target"], fold_ensemble_preds)
#         return score / len(all_folds)
    
#     def dim_opt(oof_preds_df, *args):
#         weights = args
#         score = 0
#         for fold in all_folds:
#             oof_fold_preds_df = oof_preds_df[oof_preds_df["fold"] == fold]
#             fold_ensemble_preds = np.zeros((oof_fold_preds_df.shape[0], len(weights)))
#             for idx, (weight, oof_column) in enumerate(zip(weights, oof_columns)):
#                 fold_ensemble_preds[:, idx] = weight * oof_fold_preds_df[oof_column].rank(pct=True).values
#             fold_ensemble_preds = fold_ensemble_preds.sum(axis=1) ** (1 / sum(weights))
#             score += compute_pauc(oof_fold_preds_df["target"], fold_ensemble_preds)
#         return score / len(all_folds)

    def q(**ws):
        ws = tuple(ws.values())
        return dim_opt(oof_preds_df, *ws)

    optimizer = BayesianOptimization(
        f=q,
        pbounds=pbounds,
        random_state=2022,
    )

    optimizer.maximize(
        init_points=init_points,
        n_iter=n_iter,
    )
    
    weights = []
    for i in range(len(oof_columns)):
        weights.append(optimizer.max["params"][f"w{i}"])
    
    print(f"Best pAUC: {optimizer.max['target']}")
    print(f"Best weights: {weights}")
    return weights


weights = blend_optimizer(
    oof_preds_df, oof_columns, 
    init_points=40, 
    n_iter=40
)

|   iter    |  target   |    w0     |    w1     |    w2     |
-------------------------------------------------------------
| 1         | 0.1723    | 0.09458   | 4.991     | 1.135     |
| 2         | 0.1753    | 0.5007    | 6.854     | 4.87      |
| 3         | 0.1758    | 8.977     | 6.475     | 8.97      |
| 4         | 0.1757    | 7.212     | 8.314     | 8.276     |
| 5         | 0.1733    | 8.336     | 9.57      | 3.681     |
| 6         | 0.1761    | 4.949     | 3.396     | 6.195     |
| 7         | 0.1752    | 9.775     | 0.9652    | 7.442     |
| 8         | 0.1758    | 2.926     | 2.987     | 7.525     |
| 9         | 0.1747    | 0.1876    | 5.238     | 8.644     |
| 10        | 0.176     | 3.889     | 2.123     | 4.752     |
| 11        | 0.176     | 5.647     | 3.495     | 9.759     |
| 12        | 0.1744    | 0.3792    | 7.943     | 3.579     |
| 13        | 0.1736    | 7.48      | 9.145     | 3.727     |
| 14        | 0.167     | 9.649     | 0.8148    | 0.4255    |
| 15    

In [6]:
weights

[3.30655822204979, 3.764848402784958, 6.301137081207355]

In [7]:
all_folds = np.unique(oof_preds_df["fold"])
for fold in all_folds:
    fold_index = oof_preds_df[oof_preds_df["fold"] == fold].index
    fold_target = oof_preds_df.loc[fold_index, target_column]
    fold_ensemble_preds = 0
    for model_name, version, weight in zip(model_names, versions, weights):
        fold_model_preds = oof_preds_df.loc[fold_index, f"oof_{model_name}_{version}"]
        fold_ensemble_preds += fold_model_preds.rank(pct=True).values * weight 
        
        val_auc_scores[f"{model_name}_{version}"][f"fold_{fold}"] = compute_auc(fold_target, fold_model_preds)
        val_pauc_scores[f"{model_name}_{version}"][f"fold_{fold}"] = compute_pauc(fold_target, fold_model_preds, min_tpr=0.8)
    
    oof_preds_df.loc[fold_index, ensemble_column] = fold_ensemble_preds
    val_auc_scores["ensemble"][f"fold_{fold}"] = compute_auc(fold_target, fold_ensemble_preds)
    val_pauc_scores["ensemble"][f"fold_{fold}"] = compute_pauc(fold_target, fold_ensemble_preds, min_tpr=0.8)

for model_name, version, weight in zip(model_names, versions, weights):
    print(f"Model: {model_name}_{version} | Weightage: {weight}")
    
    print("Val AUC scores:")
    pprint(val_auc_scores[f"{model_name}_{version}"])
    print("Val PAUC scores:")
    pprint(val_pauc_scores[f"{model_name}_{version}"])
    
    cv_model_auc_oof = compute_auc(oof_preds_df[target_column], oof_preds_df[f"oof_{model_name}_{version}"])
    cv_model_pauc_oof = compute_pauc(oof_preds_df[target_column], oof_preds_df[f"oof_{model_name}_{version}"], min_tpr=0.8)

    cv_model_auc_avg = np.mean(list(val_auc_scores[f"{model_name}_{version}"].values()))
    cv_model_pauc_avg = np.mean(list(val_pauc_scores[f"{model_name}_{version}"].values()))

    cv_model_auc_std = np.std(list(val_auc_scores[f"{model_name}_{version}"].values()))
    cv_model_pauc_std = np.std(list(val_pauc_scores[f"{model_name}_{version}"].values()))
    
    print(f"CV AUC OOF: {cv_model_auc_oof}")
    print(f"CV PAUC OOF: {cv_model_pauc_oof}")
    print(f"CV AUC AVG: {cv_model_auc_avg}")
    print(f"CV PAUC AVG: {cv_model_pauc_avg}")
    print(f"CV AUC STD: {cv_model_auc_std}")
    print(f"CV PAUC STD: {cv_model_pauc_std}")
    print("\n")

print("Val AUC scores:")
pprint(val_auc_scores["ensemble"])
print("Val PAUC scores:")
pprint(val_pauc_scores["ensemble"])

cv_ensemble_auc_oof = compute_auc(oof_preds_df[target_column], oof_preds_df[ensemble_column])
cv_ensemble_pauc_oof = compute_pauc(oof_preds_df[target_column], oof_preds_df[ensemble_column], min_tpr=0.8)

cv_ensemble_auc_avg = np.mean(list(val_auc_scores["ensemble"].values()))
cv_ensemble_pauc_avg = np.mean(list(val_pauc_scores["ensemble"].values()))

cv_ensemble_auc_std = np.std(list(val_auc_scores["ensemble"].values()))
cv_ensemble_pauc_std = np.std(list(val_pauc_scores["ensemble"].values()))

print(f"CV AUC OOF: {cv_ensemble_auc_oof}")
print(f"CV PAUC OOF: {cv_ensemble_pauc_oof}")
print(f"CV AUC AVG: {cv_ensemble_auc_avg}")
print(f"CV PAUC AVG: {cv_ensemble_pauc_avg}")
print(f"CV AUC STD: {cv_ensemble_auc_std}")
print(f"CV PAUC STD: {cv_ensemble_pauc_std}")

Model: cb_v1 | Weightage: 3.30655822204979
Val AUC scores:
{'fold_1': 0.9523881910669536,
 'fold_2': 0.9535923162036922,
 'fold_3': 0.9457296489498446,
 'fold_4': 0.9721251762738515,
 'fold_5': 0.9470801267252167}
Val PAUC scores:
{'fold_1': 0.1645271757829003,
 'fold_2': 0.1669476383467947,
 'fold_3': 0.15652509640463735,
 'fold_4': 0.17601833248056303,
 'fold_5': 0.15814944833701147}
CV AUC OOF: 0.9524292434775488
CV PAUC OOF: 0.16241203053404624
CV AUC AVG: 0.9541830918439118
CV PAUC AVG: 0.16443353827038137
CV AUC STD: 0.009459406837513047
CV PAUC STD: 0.006965201549198192


Model: lgb_v3 | Weightage: 3.764848402784958
Val AUC scores:
{'fold_1': 0.9576815612354752,
 'fold_2': 0.9508534742767404,
 'fold_3': 0.9506974516728857,
 'fold_4': 0.9630832636557638,
 'fold_5': 0.9616494948942257}
Val PAUC scores:
{'fold_1': 0.16882108869081178,
 'fold_2': 0.16124885048313303,
 'fold_3': 0.16251325953750728,
 'fold_4': 0.1684323170807802,
 'fold_5': 0.17159456378130328}
CV AUC OOF: 0.95469382

In [8]:
oof_preds_df

,isic_id,patient_id,fold,target,oof_cb_v1,oof_lgb_v3,oof_resnet18_v5,oof_preds_ensemble
0,ISIC_0015670,IP_1235828,4,0,0.000104,0.000015,0.000274,4.653748
1,ISIC_0015845,IP_8170065,1,0,0.456523,0.652166,0.013589,12.647800
2,ISIC_0015864,IP_6724798,5,0,0.000034,0.000019,0.000735,3.803535
3,ISIC_0015902,IP_4111386,2,0,0.000022,0.000024,0.000783,6.525818
4,ISIC_0024200,IP_8313778,1,0,0.000522,0.000160,0.003239,10.611330
...,...,...,...,...,...,...,...,...
401054,ISIC_9999937,IP_1140263,3,0,0.003126,0.002748,0.029915,12.675234
401055,ISIC_9999951,IP_5678181,3,0,0.000053,0.000030,0.000052,3.684417
401056,ISIC_9999960,IP_0076153,2,0,0.001993,0.000537,0.000115,8.396833
401057,ISIC_9999964,IP_5231513,5,0,0.000071,0.000028,0.001942,6.157597


In [9]:
oof_preds_df.pivot_table(index="target", values=oof_columns+["oof_preds_ensemble"], aggfunc="mean")

,oof_cb_v1,oof_lgb_v3,oof_preds_ensemble,oof_resnet18_v5
target,,,,
0,0.000671,0.000313,6.680433,0.012635
1,0.034607,0.021116,12.723760,0.258892


In [10]:
oof_preds_df[oof_columns+["oof_preds_ensemble"]].corr()

,oof_cb_v1,oof_lgb_v3,oof_resnet18_v5,oof_preds_ensemble
oof_cb_v1,1.000000,0.757476,0.276547,0.189871
oof_lgb_v3,0.757476,1.000000,0.216699,0.124920
oof_resnet18_v5,0.276547,0.216699,1.000000,0.391995
oof_preds_ensemble,0.189871,0.124920,0.391995,1.000000
